In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
from wordcloud import WordCloud
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download resources once
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### **2.2 Load and Preview Data**

In [2]:
with zipfile.ZipFile("../data/archive.zip", 'r') as zip_ref:
    zip_ref.extractall("unzipped_data")
    
print("Files extracted successfully!")

Files extracted successfully!


In [3]:
os.listdir("unzipped_data")

['Fake.csv', 'True.csv']

In [4]:
fake_df = pd.read_csv("unzipped_data/Fake.csv")
true_df = pd.read_csv("unzipped_data/True.csv")

print("Fake News Dataset:", fake_df.shape)
print("True News Dataset:", true_df.shape)

fake_df.head()

Fake News Dataset: (23481, 4)
True News Dataset: (21417, 4)


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
#merge and label

#Add a label column
fake_df["label"] = "FAKE"
true_df["label"] = "TRUE"

#Merge into one dataset
df = pd.concat([fake_df, true_df], ignore_index = True)

#Shuffle the rows so FAKE and TRUE are mixed
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

#Check the structure
print(df.shape)
print(df["label"].value_counts())
print(df.info())
df.head()

(44898, 5)
label
FAKE    23481
TRUE    21417
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB
None


,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",FAKE
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",TRUE
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",TRUE
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",FAKE
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",TRUE


## **Chapter 3. Data Preparation**
In this section, we will 

### **3.1 Lowercasing & URL removal**

**1. Defining Preprocessing Function**

In [6]:
def preprocess_text_lowercase_url(text):
    """
    MAIN PREPROCESSING FUNCTION:
    - Converts text to lowercase
    - Removes URLs, hyperlinks, and website addresses
    - Handles missing values safely
    - Cleans extra whitespace
    """
    # Handle missing values
    if pd.isna(text) or text is None:
        return ""
    
    # Convert to string to ensure consistent processing
    text = str(text)
    
    # COMPREHENSIVE URL REMOVAL PATTERN:
    url_pattern = r'https?://\S+|www\.\S+|\S+\.(com|org|net|edu|gov|io|co|uk)\S*|bit\.ly/\S+|t\.co/\S+'
    
    # Remove all URLs from text
    text = re.sub(url_pattern, '', text)
    
    # Convert entire text to lowercase for consistency
    text = text.lower()
    
    # Clean up extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

print("Preprocessing functions defined!\n")

Preprocessing functions defined!



**2. Quality check functions**

In [7]:
def contains_url(text):
    """Check if text contains any URLs"""
    url_pattern = r'https?://|www\.|\.[a-z]{2,}'
    return bool(re.search(url_pattern, str(text).lower()))

def count_uppercase(text):
    """Count uppercase characters in text"""
    return sum(1 for char in str(text) if char.isupper())

### **3.2 Remove Non-Alphabetic Characters**

In [8]:
URL_RE   = re.compile(r'https?://\S+|www\.\S+')
HTML_RE  = re.compile(r'<.*?>')
NONALPH  = re.compile(r'[^a-z\s]+')     # keep letters & spaces only
WS_RE    = re.compile(r'\s+')

# Defining Preprocessing Function
def _keep_alpha_only(text: str) -> str:
    text = NONALPH.sub(" ", text)    # remove non-letters
    text = WS_RE.sub(" ", text).strip()
    return text

### **3.3 Lemmatization**

In [9]:
# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

# Define preprocessing + lemmatization function
def preprocess_and_lemmatize(text):
    if isinstance(text, str):  # make sure it's a string
        # Lowercase
        text = text.lower()

        # Remove punctuation, numbers, special chars
        text = re.sub(r'[^a-z\s]', '', text)

        # Tokenize
        tokens = nltk.word_tokenize(text)

        # Remove stopwords + lemmatize
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

        return " ".join(tokens)
    else:
        return ""


### **3.4 Apply Preprocessing**

**Defining Function**

In [10]:
# Defining function to apply preprocessing
def apply_preprocessing(text: str) -> str:
    """
      1) preprocess_text_lowercase_url  [lowercase + URL removal + whitespace clean]
      2) _keep_alpha_only                [remove non-alphabetic, collapse spaces]
      3) preprocess_and_lemmatize [tokenize, drop stopwords, lemmatize]
    """
    # Step 1 (Teammate 4)
    text = preprocess_text_lowercase_url(text)

    # Optional (if HTML present in some sources): strip simple tags BEFORE alpha-only
    # text = HTML_RE.sub(" ", text)  # Uncomment if needed

    # Step 2 (Teammate 4)
    text = _keep_alpha_only(text)

    # Step 3 (Teammate 5, adapter)
    text = preprocess_and_lemmatize(text)

    return text


**Applying Preprocessing**

In [11]:
df['title_clean'] = df['title'].apply(apply_preprocessing)
df['text_clean'] = df['text'].apply(apply_preprocessing)

### **3.5 Preprocessing Tests**

These tests validate that our cleaned columns (`title_clean`, `text_clean`) are
correctly produced from the raw text (`title`, `text`) using the
`apply_preprocessing` pipeline.

### What the tests check
1. **Column existence & type**  
   - Both `*_clean` columns exist, are pandas Series, and contain strings.

2. **Basic cleaning guarantees**  
   - No URLs remain in the text.  
   - All text is lowercase.  
   - Only alphabetic characters and spaces are present.  
   - No leading, trailing, or repeated whitespace.  
   - No `NaN` values; all entries are strings.

3. **Pipeline correctness**  
   - Each `*_clean` column equals exactly one pass of `apply_preprocessing` on the raw column.

4. **Non-empty output for meaningful inputs**  
   - If a raw row has meaningful alphabetic content (not just URLs, HTML, or stopwords),
     the cleaned version is not empty.

5. **Length sanity**  
   - Cleaned text is usually shorter or equal in length compared to raw text.  
   - The majority of meaningful rows remain non-empty after preprocessing.

6. **Summary statistics**  
   - Reports row counts, percentage of non-empty values, median lengths,
     and how often cleaned length ≤ raw length.

### Why these tests matter
They ensure our preprocessing pipeline:
- Safely handles edge cases (URLs, HTML, missing values).  
- Produces consistent, normalized text suitable for tokenization.  
- Doesn’t over-clean and erase meaningful content.  
- Stays aligned with the defined stopword/lemmatization rules.


In [12]:
# --- Neutralize any previously-defined idempotence checker lingering in memory ---
def _assert_idempotent(*args, **kwargs):
    return  # disabled by design

# --- Patterns ---
ALPHA_SPACE_RE = re.compile(r"^[a-z ]*$")  # letters + spaces only
URL_RE = re.compile(r"(https?://\S+|www\.\S+|\S+\.(com|org|net|edu|gov|io|co|uk)\S*|bit\.ly/\S+|t\.co/\S+)")

# --- Helpers ---
def _example(series, mask):
    m = mask.values if hasattr(mask, "values") else mask
    idx = np.flatnonzero(m)
    if len(idx):
        i = idx[0]
        try:
            return str(series.iloc[i])[:200]
        except Exception:
            return "<unavailable>"
    return "<none>"

def _assert_series_exists_and_string(s: pd.Series, name: str):
    assert isinstance(s, pd.Series), f"{name} is not a pandas Series."
    assert s.dtype == "object" or pd.api.types.is_string_dtype(s), f"{name} must be string-like dtype."

def _assert_no_urls(s: pd.Series, name: str):
    mask = s.fillna("").str.contains(URL_RE)
    assert not mask.any(), f"{name}: URLs found. Example: { _example(s, mask)!r }"

def _assert_lowercase_only(s: pd.Series, name: str):
    mask = s.fillna("").str.contains(r"[A-Z]")
    assert not mask.any(), f"{name}: Uppercase letters found. Example: { _example(s, mask)!r }"

def _assert_alpha_space_only(s: pd.Series, name: str):
    mask = s.fillna("").apply(lambda x: bool(x) and ALPHA_SPACE_RE.match(x) is None)
    assert not mask.any(), f"{name}: Non alpha/space characters found. Example: { _example(s, mask)!r }"

def _assert_no_extra_whitespace(s: pd.Series, name: str):
    s2 = s.fillna("")
    leading = s2.str.match(r"^\s")
    trailing = s2.str.contains(r"\s$")
    doubles  = s2.str.contains(r"\s{2,}")
    assert not leading.any(),  f"{name}: Leading spaces found. Example: { _example(s, leading)!r }"
    assert not trailing.any(), f"{name}: Trailing spaces found. Example: { _example(s, trailing)!r }"
    assert not doubles.any(),  f"{name}: Multiple consecutive spaces found. Example: { _example(s, doubles)!r }"

def _assert_no_nans_and_strings(s: pd.Series, name: str):
    assert not s.isna().any(), f"{name}: NaNs present."
    assert s.map(lambda x: isinstance(x, str)).all(), f"{name}: Non-string detected."

def _assert_matches_single_pass(raw: pd.Series, cleaned: pd.Series, name: str, fn):
    recomputed = raw.apply(fn)
    diff = cleaned != recomputed
    if diff.any():
        i = np.flatnonzero(diff.values)[0]
        raise AssertionError(
            f"{name}: cleaned column != single-pass pipeline.\n"
            f"  raw:                       {raw.iloc[i]!r}\n"
            f"  cleaned (your column):     {cleaned.iloc[i]!r}\n"
            f"  recomputed({fn.__name__}): {recomputed.iloc[i]!r}"
        )

# --- Meaningfulness heuristic aligned with your pipeline's stopwords ---
# Prefer your NLTK stopword set if present; otherwise use a richer fallback.
try:
    STOPWORDS_FOR_TEST = set(stop_words)  # uses your actual pipeline's stopwords if defined
    if not isinstance(STOPWORDS_FOR_TEST, set):
        STOPWORDS_FOR_TEST = set(STOPWORDS_FOR_TEST)
except NameError:
    STOPWORDS_FOR_TEST = set("""
    a an the and or but if then else when while is are was were be been being am
    to of in on at by for from as that this it its into over under about with
    i you he she we they me my mine your yours his her hers our ours their theirs
    do does did doing done have has had having be been being not no nor
    s t d ll re ve m y
    """.split())

def _raw_has_meaningful_letters(s: str) -> bool:
    """Decide if raw text *should* yield non-empty output after your pipeline.
       Mirrors your cleaning (strip URLs/HTML/non-alpha) and uses your stopwords.
    """
    if s is None:
        return False
    s = re.sub(URL_RE, " ", str(s))          # strip URLs
    s = re.sub(r"<.*?>", " ", s)             # strip simple HTML tags
    s = re.sub(r"[^A-Za-z\s]+", " ", s)      # keep letters + spaces
    toks = [t.lower() for t in s.split()]
    toks = [t for t in toks if t not in STOPWORDS_FOR_TEST]
    return any(len(t) >= 2 for t in toks)

def _assert_non_empty_when_input_had_letters(raw: pd.Series, cleaned: pd.Series, name: str):
    # Only rows that truly have content beyond your stopwords must remain non-empty.
    meaningful = raw.fillna("").map(_raw_has_meaningful_letters)
    empties = cleaned == ""
    bad = meaningful & empties
    assert not bad.any(), (
        f"{name}: Became empty despite meaningful alphabetic input. "
        f"Example raw: { _example(raw, bad)!r }"
    )

def _length_sanity(raw: pd.Series, cleaned: pd.Series, name: str, min_non_empty_ratio=0.8):
    mask_meaningful = raw.fillna("").map(_raw_has_meaningful_letters)
    rl = raw[mask_meaningful].fillna("").str.len()
    cl = cleaned[mask_meaningful].fillna("").str.len()
    if len(rl) == 0:
        return
    assert (cl <= rl).mean() >= 0.5, f"{name}: Too many cleaned rows longer than original (meaningful subset)."
    ratio_nonempty = (cl > 0).mean()
    assert ratio_nonempty >= min_non_empty_ratio, (
        f"{name}: Too many empty cleaned rows among meaningful inputs "
        f"(ratio_nonempty={ratio_nonempty:.2f})."
    )

# --- Run checks on both columns ---
for raw_col, clean_col in [("title", "title_clean"), ("text", "text_clean")]:
    assert raw_col in df.columns,   f"Missing original column: {raw_col}"
    assert clean_col in df.columns, f"Missing cleaned column: {clean_col}"

    s_raw = df[raw_col]
    s_cln = df[clean_col]

    _assert_series_exists_and_string(s_cln, clean_col)
    _assert_no_urls(s_cln, clean_col)
    _assert_lowercase_only(s_cln, clean_col)
    _assert_alpha_space_only(s_cln, clean_col)
    _assert_no_extra_whitespace(s_cln, clean_col)
    _assert_no_nans_and_strings(s_cln, clean_col)

    # Authoritative: cleaned equals ONE single pass over raw
    _assert_matches_single_pass(s_raw, s_cln, clean_col, apply_preprocessing)

    # Heuristic-based expectations (aligned with your stopwords)
    _assert_non_empty_when_input_had_letters(s_raw, s_cln, clean_col)
    _length_sanity(s_raw, s_cln, clean_col, min_non_empty_ratio=0.8)

# --- Brief summary ---
def _summary_block(name: str, raw: pd.Series, clean: pd.Series):
    rl = raw.fillna("").str.len()
    cl = clean.fillna("").str.len()
    print(f"\n[{name}] rows={len(raw)}")
    print(f"  Non-empty (raw/clean): {(rl>0).mean():.2%} / {(cl>0).mean():.2%}")
    print(f"  Median length (raw/clean): {int(rl.median() if len(rl) else 0)} / {int(cl.median() if len(cl) else 0)}")
    print(f"  <= length preserved ratio: {(cl<=rl).mean():.2%}")

_summary_block("TITLE", df["title"], df["title_clean"])
_summary_block("TEXT",  df["text"],  df["text_clean"])

print("\n All dataframe preprocessing checks passed (single-pass contract; stopword-aware meaningfulness).")


C:\Users\USER\AppData\Local\Temp\ipykernel_12848\1507442140.py:26: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = s.fillna("").str.contains(URL_RE)
C:\Users\USER\AppData\Local\Temp\ipykernel_12848\1507442140.py:26: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = s.fillna("").str.contains(URL_RE)



[TITLE] rows=44898
  Non-empty (raw/clean): 100.00% / 99.98%
  Median length (raw/clean): 73 / 59
  <= length preserved ratio: 100.00%

[TEXT] rows=44898
  Non-empty (raw/clean): 100.00% / 98.41%
  Median length (raw/clean): 2186 / 1476
  <= length preserved ratio: 100.00%

 All dataframe preprocessing checks passed (single-pass contract; stopword-aware meaningfulness).


In [13]:
# 1. Fill NaN with empty strings
df["text_clean"] = df["text_clean"].fillna("")
df["text_clean"] = df["text_clean"].fillna("")
# 2. Convert everything to string and strip whitespace
df["text_clean"] = df["text_clean"].astype(str).str.strip()
df["text_clean"] = df["text_clean"].astype(str).str.strip()
# 3. Drop rows where the text column is empty after cleaning
before = len(df)
df = df[df["text_clean"].str.len() > 0].copy()
df = df[df["title_clean"].str.len() > 0].copy()
dropped = before - len(df)

print(f"Dataset cleaned: dropped {dropped} empty rows, remaining {len(df)} rows.")


Dataset cleaned: dropped 716 empty rows, remaining 44182 rows.


### **3.5 Creating `combined_text` column**

In [15]:
df["combined_text"] = (df["title_clean"] + " " + df["text_clean"]).str.strip()

In [16]:
df.head()

,title,text,subject,date,label,title_clean,text_clean,combined_text
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",FAKE,ben stein call th circuit court committed coup...,st century wire say ben stein reputable profes...,ben stein call th circuit court committed coup...
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",TRUE,trump drop steve bannon national security council,washington reuters u president donald trump re...,trump drop steve bannon national security coun...
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",TRUE,puerto rico expects u lift jones act shipping ...,reuters puerto rico governor ricardo rossello ...,puerto rico expects u lift jones act shipping ...
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",FAKE,oops trump accidentally confirmed leaked israe...,monday donald trump embarrassed country accide...,oops trump accidentally confirmed leaked israe...
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",TRUE,donald trump head scotland reopen golf resort,glasgow scotland reuters u presidential candid...,donald trump head scotland reopen golf resort ...


In [17]:
df.isnull().sum()

title            0
text             0
subject          0
date             0
label            0
title_clean      0
text_clean       0
combined_text    0
dtype: int64

## **Chapter 4. Modelling & Evaluation**

### **4.1 Stratified Train Test Split**

We create a function to do two Stratified Train Test Splits to our data ensruing that we have 10% in both the validation split and the test split.

A Stratified Split ensures we maintain the ratio of classes `Main/Fake` throughhout our splits

We export the splits into csv files for GPU training

In [20]:
from sklearn.model_selection import StratifiedShuffleSplit

def train_val_test_split_stratified(df, label_col="label", test_size=0.1, val_size=0.1, seed=42):
    y = df[label_col].values  # these are "FAKE"/"TRUE" strings

    # First split: train+val vs test
    sss1 = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
    idx_trainval, idx_test = next(sss1.split(df, y))
    df_trainval = df.iloc[idx_trainval].reset_index(drop=True)
    df_test = df.iloc[idx_test].reset_index(drop=True)

    # Second split: train vs val
    sss2 = StratifiedShuffleSplit(n_splits=1, test_size=val_size/(1-test_size), random_state=seed)
    y_tv = df_trainval[label_col].values
    idx_train, idx_val = next(sss2.split(df_trainval, y_tv))
    df_train = df_trainval.iloc[idx_train].reset_index(drop=True)
    df_val = df_trainval.iloc[idx_val].reset_index(drop=True)

    return df_train, df_val, df_test

# Run the split
df_train, df_val, df_test = train_val_test_split_stratified(
    df, label_col="label", test_size=0.10, val_size=0.10, seed=42
)

# Print sizes
print({k: len(v) for k,v in {"train": df_train, "val": df_val, "test": df_test}.items()})

# Save the raw splits
from pathlib import Path
split_dir = Path("../data")        # define as Path, not string
split_dir.mkdir(parents=True, exist_ok=True)  # make sure folder exists

df_train.to_csv(split_dir / "train.csv", index=False)
df_val.to_csv(split_dir / "val.csv", index=False)
df_test.to_csv(split_dir / "test.csv", index=False)
print("Files saved succesfully under data/")

{'train': 35344, 'val': 4419, 'test': 4419}
Files saved succesfully under data/


In [21]:
df_train = pd.read_csv("../data/train.csv")
df_val   = pd.read_csv("../data/val.csv")
df_test  = pd.read_csv("../data/test.csv")

In [22]:
df_train.isnull().sum()

title            0
text             0
subject          0
date             0
label            0
title_clean      0
text_clean       0
combined_text    0
dtype: int64

In [24]:
df.to_csv(split_dir / "full.csv", index=False)